In [1]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime 

In [2]:
folder_path = './dataset/'
randomtrain_folder_path = "./randomtrain/"
randomtest_folder_path = "./randomtest/"
# firsttrain_folder_path = "./firsttrain/"
# lasttest_folder_path = "./lasttest/"
# lastrandomtest_folder_path = "./lastrandomtest/"


file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]


if not os.path.exists(randomtrain_folder_path):
    os.makedirs(randomtrain_folder_path)
if not os.path.exists(randomtest_folder_path):
    os.makedirs(randomtest_folder_path)
# if not os.path.exists(firsttrain_folder_path):
#     os.makedirs(firsttrain_folder_path)
# if not os.path.exists(lasttest_folder_path):
#     os.makedirs(lasttest_folder_path)
# if not os.path.exists(lastrandomtest_folder_path):
#     os.makedirs(lastrandomtest_folder_path)


In [3]:
#split the data and make sure the proportion in train and test set are equal
def getRandomSplit(data, size):
    from sklearn.model_selection import StratifiedShuffleSplit
    split = StratifiedShuffleSplit(n_splits = 1,test_size = size,random_state = 42)

    for train_index,test_index in split.split(data,data.iloc[:,-1]):
        train_set = data.iloc[train_index,:]
        test_set = data.iloc[test_index,:]

    print(len(train_set),len(test_set))
    return train_set,test_set

In [4]:
#split the last 10% data 

# window
def sliding_window(data, window_size,anomaly_ratio):
    # from last raw
    for i in range(len(data)-1, window_size-2, -10):
        data_slice = data[i-window_size+1:i+1]
        ratio = (data_slice["Label"] == 1).sum() / window_size*100
        #print(ratio)
        if anomaly_ratio*0.8 <= ratio <= anomaly_ratio*1.2:
            #print(data_slice)
            return data_slice


def getLastSplit(data, size,anomaly_ratio):
    data.sort_values("datetime", ascending=True, inplace=True)
    #print(data)
    row,col=data.shape
    window_size = int(size*row)
    test_set = sliding_window(data, window_size,anomaly_ratio)
    if test_set is None:
        print("no test")
    time = test_set.iloc[0,0]
    #print(time)
    train_set = data[data['datetime'] < time]
    print(len(train_set),len(test_set))
    return train_set,test_set
# windows keep the same rate of anomaly

In [5]:
#def compute ratio
def comp_ratio(test_set1):
    instance = test_set1[(test_set1['Label']==0)]
    rate=len(instance)/len(test_set1)*100
    print(rate)
    return rate

In [8]:
files = [#"PR_PF_4903220.csv", "PR_PF_6901195.csv", "PR_PF_3901684.csv", "PR_PF_3901857.csv",
         #"PR_PF_6901272.csv", "PR_PF_4902338.csv", "PR_PF_4902337.csv","PR_PF_4903217.csv" ,
    #"PR_PF_4903218.csv", "PR_PF_4903216.csv"
    "PR_PF_4902919.csv"
]


for file_name in files:
    print(file_name)
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path)
    anomaly_ratio=comp_ratio(data)
    train_set1,test_set1 = getRandomSplit(data,0.3)# random 70 30
        #train_set2,test_set2 = getLastSplit(data,0.1, anomaly_ratio) #last 10 and rest
        #print("test2 length ",len(test_set2))
    comp_ratio(test_set1)
    comp_ratio(train_set1)
        #comp_ratio(test_set2)
        #rate=len(test_set2)/len(train_set2)
        #print("rate",rate)
        #train_set22,test_set22=getRandomSplit(train_set2, rate) #first 
        #print("test22 length", len(test_set22))
        #comp_ratio(test_set22)
        #keep the same train size
        #train_set1 = train_set1.sample(n=len(train_set22))
    print("train1 length",len(train_set1))
    train_set1.to_csv(os.path.join(randomtrain_folder_path, f"{file_name}"), index=False)
    test_set1.to_csv(os.path.join(randomtest_folder_path, f"{file_name}"), index=False)
    print("==============Done==============")

PR_PF_4902919.csv
85.6062280952916
193137 82774
85.60659144175708
85.60607237349653
train1 length 193137
==============Done==============
